<a href="https://colab.research.google.com/github/Christian-Divinefavour/Titanic---Machine-Learning-from-Disaster/blob/main/TITANIC_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [2]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Activation, Flatten
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train_df.set_index(['PassengerId'], inplace = True)
test_df.set_index(['PassengerId'], inplace = True)

In [6]:
#Let's generate random numbers between mean and std of Age to replace null values
train_mn = train_df['Age'].mean()
train_sd = train_df['Age'].std()
import random
from random import randrange
for i in range(177):
    val = random.uniform(train_sd, train_mn)
train_df['Age'].replace(np.nan, val, inplace = True)
train_df['Age']

PassengerId
1      22.000000
2      38.000000
3      26.000000
4      35.000000
5      35.000000
         ...    
887    27.000000
888    19.000000
889    19.201714
890    26.000000
891    32.000000
Name: Age, Length: 891, dtype: float64

In [7]:
test_mn = test_df['Age'].mean()
test_sd = test_df['Age'].std()
import random
from random import randrange
for i in range(test_df['Age'].isnull().sum()):
    tval = random.uniform(test_sd, test_mn)
test_df['Age'].replace(np.nan, tval, inplace = True)
test_df['Age']

PassengerId
892     34.500000
893     47.000000
894     62.000000
895     27.000000
896     22.000000
          ...    
1305    20.199917
1306    39.000000
1307    38.500000
1308    20.199917
1309    20.199917
Name: Age, Length: 418, dtype: float64

In [8]:
#Drop cabin column in both test and train cos of excess null values
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis =1)
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [9]:
#Not much correlation. Drop fare column
train_df = train_df.drop(['Fare'], axis = 1)
test_df = test_df.drop(['Fare'], axis = 1)

In [10]:
#Convert cateorical to numerical in Sex
train_df['Sex']=train_df['Sex'].map({'female' : 1,'male' : 0}).astype(int)
test_df['Sex']=test_df['Sex'].map({'female' : 1,'male' : 0}).astype(int)

In [11]:
train_df['Embarked'].value_counts()
train_df.Embarked.replace(np.nan, 'S', inplace = True)
dummy1 = pd.get_dummies(train_df['Embarked'])
train_df = pd.concat([train_df, dummy1], axis = 1)
dummy2 = pd.get_dummies(test_df['Embarked'])
test_df = pd.concat([test_df, dummy2], axis = 1)

In [12]:
train_df = train_df.drop('Embarked', axis = 1)
test_df = test_df.drop('Embarked', axis =1)

In [13]:
train_df = train_df.drop('Ticket', axis = 1)
test_df = test_df.drop('Ticket', axis =1)

In [14]:
train_df['Age'] = train_df['Age'].astype(int)
test_df['Age'] = test_df['Age'].astype(int)

In [15]:
train_df.loc[ train_df['Age'] <= 16, 'Age'] = 0
train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 36), 'Age'] = 1
train_df.loc[(train_df['Age'] > 36) & (train_df['Age'] <= 50), 'Age'] = 2
train_df.loc[(train_df['Age'] > 50) & (train_df['Age'] <= 64), 'Age'] = 3
train_df.loc[ train_df['Age'] > 64, 'Age'] = 4

test_df.loc[ test_df['Age'] <= 16, 'Age'] = 0
test_df.loc[(test_df['Age'] > 16) & (test_df['Age'] <= 36), 'Age'] = 1
test_df.loc[(test_df['Age'] > 36) & (test_df['Age'] <= 50), 'Age'] = 2
test_df.loc[(test_df['Age'] > 50) & (test_df['Age'] <= 64), 'Age'] = 3
test_df.loc[ test_df['Age'] > 64, 'Age'] = 4

In [16]:
train_df = train_df.drop('Name', axis = 1)
test_df = test_df.drop('Name', axis = 1)

In [17]:
Y_train = train_df['Survived']
X_train = train_df.drop(['Survived'], axis = 1)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.8)

In [19]:
model = tf.keras.Sequential([layers.Dense(units=1, input_shape = [len(X_train.columns)], activation = 'relu')])

In [20]:
len(X_train.columns)

8

In [21]:
model.add(tf.keras.layers.Dense(128, activation= 'relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation= 'linear'))

In [22]:
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 9         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               256       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 16,906
Trainable params: 16,906
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(x_train, y_train, batch_size=10, epochs=200)

Epoch 1/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4194
Epoch 2/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4168
Epoch 3/200
18/18 [==============================] - 0s 1ms/step - loss: 0.4184
Epoch 4/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4162
Epoch 5/200
18/18 [==============================] - 0s 1ms/step - loss: 0.4198
Epoch 6/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4164
Epoch 7/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4165
Epoch 8/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4158
Epoch 9/200
18/18 [==============================] - 0s 1ms/step - loss: 0.4153
Epoch 10/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4135
Epoch 11/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4162
Epoch 12/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4148
Epoch 13/200
18/18 [=================

In [24]:
pp = model.predict(x_test)
MAE = mean_absolute_error(y_test, pp)
MAE

0.2592146932493587

In [25]:
pred = model.predict(test_df)
pred = pred.astype(int)
pred = pd.DataFrame(pred, columns=['Survived'])
pred.head()

,Survived
0,0
1,0
2,0
3,0
4,0


In [26]:
pred

,Survived
0,0
1,0
2,0
3,0
4,0
...,...
413,0
414,0
415,0
416,0


In [27]:
test1 = pd.read_csv("test.csv")
submission = pd.DataFrame(test1['PassengerId'])
submission

,PassengerId
0,892
1,893
2,894
3,895
4,896
...,...
413,1305
414,1306
415,1307
416,1308


In [28]:
submission = pd.concat([submission, pred], axis = 1)
submission.set_index(['PassengerId'], inplace = True)
submission

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
...,...
1305,0
1306,0
1307,0


In [29]:
submission.to_csv('Submit.csv')